In [0]:
## Cleaning of comments

import re
def clean_comments(text): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
ct = []
for t in range(len(reviews)):
    ct.append(clean_comments(reviews[t]))
    
data1['Clear_Comments']=ct


## converting text into vectors 

import nltk
nltk.download('stopwords')

comments = []
for a in range(len(data1['Clear_Comments'])):
    comments.append(data1['Clear_Comments'][a].lower())

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(comments).toarray()


##Data splitting into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_features, data1['sentiment'], test_size = 0.20)


## Apply SVM
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
model = svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

## Evaluation of SVM results
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
from math import sqrt

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(model.score(X_train, y_train))
print("RMSE", sqrt(mean_squared_error(y_test,y_pred)))
print("MAE", mean_absolute_error(y_test, y_pred))



### Gaussian Naive Bayes 

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)


### evaluation of naive bayes

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
from math import sqrt

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(gnb.score(X_train, y_train))
print("RMSE", sqrt(mean_squared_error(y_test,p)))
print("MAE", mean_absolute_error(y_test, p))


## Convolutional Neural network
### do not perform vectorization for CNN

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

### tokenize

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

## finds the vocabulary size and then perform padding on both train and test set

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()


embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

### CNN
from keras.layers import Conv1D
model = Sequential()

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())


history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

## predicted values
y_p = model.predict_classes(X_test)
y_pred = np.array(y_p).astype(int)

## evaluation of CNN

import numpy
import tensorflow as tf
tf.executing_eagerly()

m = tf.keras.metrics.Accuracy() 
_ = m.update_state(y_test, y_pred)
m.result().numpy()

m = tf.keras.metrics.MeanAbsoluteError() 
_ = m.update_state(y_test, y_pred) 
m.result().numpy()

m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred)
print('Precision: ', m.result().numpy())

m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred)
print('Recall: ', m.result().numpy())

m = tf.keras.metrics.RootMeanSquaredError()
m.update_state(y_test, y_pred)
print('RMSE: ', m.result().numpy())


### plot of accuracy and loss

import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()